In [7]:
# Install required libraries in a Jupyter Notebook environment
!pip install requests pandas numpy datetime scikit-learn plotly holidays

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [8]:
import pandas as pd
# =============================================================================
# 1. LOAD THE DATA
# =============================================================================
# Replace these file paths with your actual CSV paths
df_2018 = pd.read_csv('YEAR/sessions_filtered_2018.csv')
df_2019 = pd.read_csv('YEAR/sessions_filtered_2019.csv')
df_2020 = pd.read_csv('YEAR/sessions_filtered_2020.csv')
df_2021 = pd.read_csv('YEAR/sessions_filtered_2021.csv')


In [9]:
import pandas as pd
import plotly.express as px
import holidays

# =============================================================================
# 1. Load Raw Data
# =============================================================================

# Replace these file paths with actual file paths
df_2018 = pd.read_csv('YEAR/sessions_filtered_2018.csv')
df_2019 = pd.read_csv('YEAR/sessions_filtered_2019.csv')
df_2020 = pd.read_csv('YEAR/sessions_filtered_2020.csv')
df_2021 = pd.read_csv('YEAR/sessions_filtered_2021.csv')


# =============================================================================
# 2. Session Data Preprocessing Function
# =============================================================================

def process_sessions_df(df_raw, year_label):
    """Clean and process a raw sessions DataFrame."""
    # Drop rows with missing key data
    df = df_raw.dropna(subset=[
        'sessionID', 'connectionTime', 'disconnectTime', 'doneChargingTime', 'kWhDelivered'
    ]).copy()

    # Convert timestamps to datetime
    for col in ['connectionTime', 'disconnectTime', 'doneChargingTime']:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # Drop rows with failed datetime conversions
    df.dropna(subset=['connectionTime', 'disconnectTime'], inplace=True)

    # Compute session duration (in hours)
    df['session_duration'] = (
        (df['disconnectTime'] - df['connectionTime']).dt.total_seconds() / 3600.0
    )

    # Filter durations between 0 and 12 hours
    df = df[(df['session_duration'] > 0) & (df['session_duration'] < 12)]

    # Compute charging duration (in hours)
    df['charge_duration'] = (
        (df['doneChargingTime'] - df['connectionTime']).dt.total_seconds() / 3600.0
    )
    df = df[df['charge_duration'] > 0]

    # Calculate average power output (kW)
    df['power_output'] = df['kWhDelivered'] / df['charge_duration']

    # Add year label
    df['year'] = year_label

    return df


# =============================================================================
# 3. Process All Years and Combine
# =============================================================================

df1 = process_sessions_df(df_2018, 2018)
df2 = process_sessions_df(df_2019, 2019)
df3 = process_sessions_df(df_2020, 2020)
df4 = process_sessions_df(df_2021, 2021)

df = pd.concat([df1, df2, df3, df4], ignore_index=True)


# =============================================================================
# 4. Create Power Events and Calculate Daily Average Demand
# =============================================================================

# Create start and end power events
df_start = df[['connectionTime', 'power_output']].copy()
df_start.rename(columns={'connectionTime': 'timestamp'}, inplace=True)

df_end = df[['doneChargingTime', 'power_output']].copy()
df_end.rename(columns={'doneChargingTime': 'timestamp'}, inplace=True)
df_end['power_output'] *= -1  # negate to indicate power ending

# Combine and sort events
events_df = pd.concat([df_start, df_end], ignore_index=True)
events_df.sort_values('timestamp', inplace=True)

# Cumulative power to simulate system load
events_df['total_power'] = events_df['power_output'].cumsum()

# Set timestamp as index for resampling
events_df.set_index('timestamp', inplace=True)

# Resample to daily mean system load
daily_avg = events_df['total_power'].resample('D').mean().reset_index()


# =============================================================================
# 5. Filter by Business Days and Exclude Holidays
# =============================================================================

# Exclude weekends
daily_avg = daily_avg[daily_avg['timestamp'].dt.dayofweek < 5]

# Exclude US holidays
years = [2018, 2019, 2020, 2021]
us_holidays = holidays.US(years=years)
holiday_dates = pd.to_datetime(list(us_holidays.keys()))
daily_avg = daily_avg[~daily_avg['timestamp'].isin(holiday_dates)]

# Exclude December 24–31 (any year)
daily_avg = daily_avg[
    ~((daily_avg['timestamp'].dt.month == 12) & (daily_avg['timestamp'].dt.day >= 24))
]


# =============================================================================
# 6. Plot Average Demand
# =============================================================================

fig_demand = px.scatter(
    daily_avg,
    x='timestamp',
    y='total_power',
    labels={
        'timestamp': 'Date',
        'total_power': 'Average System Demand (kW)'
    },
    title='Average kW Demand (Weekdays, Non-Holidays, Excluding Dec 24–31)'
)
fig_demand.show()


In [10]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score


def plot_nn_predictions(daily_avg, start_date, end_date, plot_start=None, plot_end=None):
    """
    Plots actual and predicted daily average kW demand using:
        - Neural network regression
        - Linear regression (degree 1)
        - Quadratic regression (degree 2)
    over a specified time period.
    """
    # ==========================================================================
    # 1. Filter and prepare training data
    # ==========================================================================
    mask = (
        (daily_avg['timestamp'] >= pd.to_datetime(start_date)) &
        (daily_avg['timestamp'] <= pd.to_datetime(end_date))
    )
    df_filtered = daily_avg.loc[mask].copy()
    df_filtered.dropna(subset=['total_power'], inplace=True)
    df_filtered.sort_values('timestamp', inplace=True)

    df_filtered['days'] = (df_filtered['timestamp'] - pd.to_datetime(start_date)).dt.days

    X = df_filtered['days'].values.reshape(-1, 1)
    y = df_filtered['total_power'].values

    # ==========================================================================
    # 2. Define prediction range
    # ==========================================================================
    plot_start_date = pd.to_datetime(plot_start or start_date)
    plot_end_date = pd.to_datetime(plot_end or end_date)

    plot_start_day = (plot_start_date - pd.to_datetime(start_date)).days
    plot_end_day = (plot_end_date - pd.to_datetime(start_date)).days

    x_range = np.linspace(plot_start_day, plot_end_day, 200).reshape(-1, 1)
    x_date_range = pd.to_datetime(start_date) + pd.to_timedelta(x_range.flatten(), unit='D')

    # ==========================================================================
    # 3. Neural network regression
    # ==========================================================================
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    x_range_scaled = scaler.transform(x_range)

    nn_model = MLPRegressor(
        hidden_layer_sizes=(10,),
        activation='relu',
        solver='adam',
        max_iter=200_000,
        learning_rate_init=0.0001,
        random_state=42,
        verbose=True
    )
    nn_model.fit(X_scaled, y)

    y_pred_nn_train = nn_model.predict(X_scaled)
    r2_nn = r2_score(y, y_pred_nn_train)
    mse_nn = mean_squared_error(y, y_pred_nn_train)

    y_pred_nn_range = nn_model.predict(x_range_scaled)

    # ==========================================================================
    # 4. Linear regression (degree 1)
    # ==========================================================================
    lin_model = LinearRegression()
    lin_model.fit(X, y)

    y_pred_lin_train = lin_model.predict(X)
    r2_lin = r2_score(y, y_pred_lin_train)
    mse_lin = mean_squared_error(y, y_pred_lin_train)

    y_pred_lin_range = lin_model.predict(x_range)

    # ==========================================================================
    # 5. Quadratic regression (degree 2)
    # ==========================================================================
    poly = PolynomialFeatures(degree=2)
    X_poly = poly.fit_transform(X)

    quad_model = LinearRegression()
    quad_model.fit(X_poly, y)

    y_pred_quad_train = quad_model.predict(X_poly)
    r2_quad = r2_score(y, y_pred_quad_train)
    mse_quad = mean_squared_error(y, y_pred_quad_train)

    x_range_poly = poly.transform(x_range)
    y_pred_quad_range = quad_model.predict(x_range_poly)

    # ==========================================================================
    # 6. Plotting
    # ==========================================================================
    fig = go.Figure()

    # Plot all data for context
    df_all = daily_avg.dropna(subset=['total_power']).sort_values('timestamp')
    fig.add_trace(go.Scatter(
        x=df_all['timestamp'],
        y=df_all['total_power'],
        mode='markers',
        name='All Data Points',
        marker=dict(color='lightblue')
    ))

    # Plot training data
    fig.add_trace(go.Scatter(
        x=df_filtered['timestamp'],
        y=df_filtered['total_power'],
        mode='markers',
        name='Training Data',
        marker=dict(size=8, color='blue')
    ))

    # Plot NN prediction
    fig.add_trace(go.Scatter(
        x=x_date_range,
        y=y_pred_nn_range,
        mode='lines',
        name=f"NN (R²={r2_nn:.3f}, MSE={mse_nn:.2f})",
        line=dict(width=3)
    ))

    # Plot linear regression
    fig.add_trace(go.Scatter(
        x=x_date_range,
        y=y_pred_lin_range,
        mode='lines',
        name=f"Linear (R²={r2_lin:.3f}, MSE={mse_lin:.2f})",
        line=dict(width=2, dash='dot')
    ))

    # Plot quadratic regression
    fig.add_trace(go.Scatter(
        x=x_date_range,
        y=y_pred_quad_range,
        mode='lines',
        name=f"Quadratic (R²={r2_quad:.3f}, MSE={mse_quad:.2f})",
        line=dict(width=2, dash='dash')
    ))

    fig.update_layout(
        title="NN vs. Linear vs. Quadratic Regression on Daily Average kW",
        xaxis_title="Date",
        yaxis_title="Average System Demand (kW)",
        legend_title="Legend"
    )

    # ==========================================================================
    # 7. Add horizontal average demand line (pre-COVID)
    # ==========================================================================
    mask_avg = (
        (daily_avg['timestamp'] >= pd.to_datetime('2018-04-25')) &
        (daily_avg['timestamp'] <= pd.to_datetime('2020-03-01'))
    )
    avg_value = daily_avg.loc[mask_avg, 'total_power'].mean()
    legend_label = f"Avg Demand pre-COVID ({avg_value:.1f} kW)"

    fig.add_shape(
        type="line",
        xref="paper",
        yref="y",
        x0=0,
        x1=1,
        y0=avg_value,
        y1=avg_value,
        line=dict(dash="dash", color="magenta", width=2)
    )
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='lines',
        line=dict(dash='dash', color='magenta', width=2),
        name=legend_label,
        showlegend=True
    ))

    # ==========================================================================
    # 8. Add expected NN recovery line (intersection with avg)
    # ==========================================================================
    diffs = y_pred_nn_range - avg_value
    i_closest = np.argmin(np.abs(diffs))
    x_intersect = x_date_range[i_closest]

    fig.add_shape(
        type="line",
        xref="x",
        yref="paper",
        x0=x_intersect,
        x1=x_intersect,
        y0=0,
        y1=1,
        line=dict(dash="dot", color="black")
    )
    fig.add_annotation(
        x=x_intersect,
        y=1.1,
        xref="x",
        yref="paper",
        text="Expected Recovery Time (NN)",
        showarrow=False,
        font=dict(color="black"),
        yanchor="top"
    )

    # ==========================================================================
    # 9. Mark COVID start line
    # ==========================================================================
    covid_start = pd.to_datetime('2020-01-13')
    fig.add_shape(
        type="line",
        xref="x",
        yref="paper",
        x0=covid_start,
        x1=covid_start,
        y0=0,
        y1=1,
        line=dict(dash="dot", color="red")
    )
    fig.add_annotation(
        x=covid_start,
        y=1.1,
        xref="x",
        yref="paper",
        text="COVID Impact",
        showarrow=False,
        font=dict(color="red"),
        yanchor="top"
    )

    # ==========================================================================
    # 10. Show and print results
    # ==========================================================================
    fig.show()
    print(f"Expected recovery date (NN crosses avg demand): {x_intersect.date()}")


# Example usage
plot_nn_predictions(
    daily_avg,
    start_date='2020-01-13',
    end_date='2021-12-31',
    plot_start='2020-01-13',
    plot_end='2023-01-01'
)


Iteration 1, loss = 79.91260758
Iteration 2, loss = 79.90312663
Iteration 3, loss = 79.89336529
Iteration 4, loss = 79.88417230
Iteration 5, loss = 79.87447370
Iteration 6, loss = 79.86491457
Iteration 7, loss = 79.85532193
Iteration 8, loss = 79.84603264
Iteration 9, loss = 79.83644120
Iteration 10, loss = 79.82696399
Iteration 11, loss = 79.81768900
Iteration 12, loss = 79.80794773
Iteration 13, loss = 79.79860919
Iteration 14, loss = 79.78897291
Iteration 15, loss = 79.77964563
Iteration 16, loss = 79.77017136
Iteration 17, loss = 79.76072497
Iteration 18, loss = 79.75119687
Iteration 19, loss = 79.74187499
Iteration 20, loss = 79.73227570
Iteration 21, loss = 79.72293509
Iteration 22, loss = 79.71326688
Iteration 23, loss = 79.70414923
Iteration 24, loss = 79.69455916
Iteration 25, loss = 79.68544831
Iteration 26, loss = 79.67543277
Iteration 27, loss = 79.66633126
Iteration 28, loss = 79.65657657
Iteration 29, loss = 79.64753275
Iteration 30, loss = 79.63799237
Iteration 31, loss 

Expected recovery date (NN crosses avg demand): 2022-01-17
